# Importing Data

In [1]:
import pandas as pd

# Load the dataset
data = pd.read_csv('accepted_2007_to_2018Q4.csv')

# Display basic information
print(data.info())
print(data.head())


C:\Users\91703\AppData\Local\Temp\ipykernel_20716\3415062341.py:4: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('accepted_2007_to_2018Q4.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Columns: 151 entries, id to settlement_term
dtypes: float64(113), object(38)
memory usage: 2.5+ GB
None
         id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  68407277        NaN     3600.0       3600.0           3600.0   36 months   
1  68355089        NaN    24700.0      24700.0          24700.0   36 months   
2  68341763        NaN    20000.0      20000.0          20000.0   60 months   
3  66310712        NaN    35000.0      35000.0          35000.0   60 months   
4  68476807        NaN    10400.0      10400.0          10400.0   60 months   

   int_rate  installment grade sub_grade  ... hardship_payoff_balance_amount  \
0     13.99       123.03     C        C4  ...                            NaN   
1     11.99       820.28     C        C1  ...                            NaN   
2     10.78       432.66     B        B4  ...                            NaN   
3     14.85       829

In [2]:
data = data.sample(frac=0.1, random_state=42)  # 10% of the data

In [3]:
print("Unique values in 'loan_status':", data['loan_status'].unique())

Unique values in 'loan_status': ['Current' 'Fully Paid' 'Charged Off' 'Late (31-120 days)'
 'Late (16-30 days)' 'In Grace Period'
 'Does not meet the credit policy. Status:Fully Paid'
 'Does not meet the credit policy. Status:Charged Off' nan 'Default']


In [4]:
data.isnull().sum()

id                            0
member_id                226070
loan_amnt                     4
funded_amnt                   4
funded_amnt_inv               4
                          ...  
settlement_status        222601
settlement_date          222601
settlement_amount        222601
settlement_percentage    222601
settlement_term          222601
Length: 151, dtype: int64

Drop columns with a high percentage of missing values

In [5]:
threshold = 0.5  # 50% threshold
data = data.loc[:, data.isnull().mean() < threshold]

In [6]:
data.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
392949,39651438,32000.0,32000.0,32000.0,60 months,10.49,687.65,B,B3,Public Service,...,28.6,0.0,0.0,556496.0,103647.0,64100.0,72197.0,N,Cash,N
1273506,16411620,9600.0,9600.0,9600.0,36 months,12.99,323.42,C,C1,NaN,...,50.0,1.0,0.0,11600.0,4509.0,2400.0,0.0,N,Cash,N
324024,45122316,4000.0,4000.0,4000.0,36 months,6.68,122.93,A,A3,System Analyst,...,0.0,0.0,0.0,222616.0,64253.0,5600.0,76154.0,N,Cash,N
2066630,125356772,6025.0,6025.0,6025.0,36 months,10.91,197.00,B,B4,Admin assistant,...,0.0,0.0,0.0,32227.0,5559.0,11000.0,11127.0,N,Cash,N
477199,128490686,25000.0,25000.0,25000.0,60 months,26.30,752.96,E,E5,Coordinator,...,10.0,0.0,0.0,257219.0,97647.0,179400.0,65719.0,N,Cash,N


In [7]:
data.isnull().sum()

id                               0
loan_amnt                        4
funded_amnt                      4
funded_amnt_inv                  4
term                             4
                              ... 
total_bc_limit                5024
total_il_high_credit_limit    6983
hardship_flag                    4
disbursement_method              4
debt_settlement_flag             4
Length: 107, dtype: int64

Fill remaining missing values 

In [8]:
#Numeric Values
data.fillna(data.median(numeric_only=True), inplace=True)

In [9]:
#Non-numeric Values
data.fillna(data.mode().iloc[0], inplace=True)

C:\Users\91703\AppData\Local\Temp\ipykernel_20716\2927964412.py:2: UserWarning: Unable to sort modes: '<' not supported between instances of 'int' and 'str'
  data.fillna(data.mode().iloc[0], inplace=True)


In [10]:
data.drop_duplicates(inplace=True)


In [11]:
#Verify if all values are filled
data.isnull().sum()

id                            0
loan_amnt                     0
funded_amnt                   0
funded_amnt_inv               0
term                          0
                             ..
total_bc_limit                0
total_il_high_credit_limit    0
hardship_flag                 0
disbursement_method           0
debt_settlement_flag          0
Length: 107, dtype: int64

In [12]:
print(data.info())


<class 'pandas.core.frame.DataFrame'>
Index: 226070 entries, 392949 to 118555
Columns: 107 entries, id to debt_settlement_flag
dtypes: float64(83), object(24)
memory usage: 186.3+ MB
None


# Feature Engineering

In [13]:
data['loan_to_income'] = data['loan_amnt'] / data['annual_inc']

In [14]:
def categorize_rate(rate):
    if rate < 10:
        return 'Low'
    elif 10 <= rate <= 15:
        return 'Medium'
    else:
        return 'High'

data['interest_rate_category'] = data['int_rate'].apply(categorize_rate)


In [15]:
data['term_numeric'] = data['term'].str.extract('(\d+)').astype(float)

In [16]:
data['debt_to_income'] = data['installment'] / data['annual_inc']

In [17]:
data['payment_to_loan_ratio'] = data['installment'] / data['loan_amnt']

In [18]:
data['credit_utilization'] = data['revol_bal'] / data['revol_util']

In [19]:
data['issue_d'] = pd.to_datetime(data['issue_d'])
data['loan_age_months'] = (pd.to_datetime('today') - data['issue_d']).dt.days / 30

C:\Users\91703\AppData\Local\Temp\ipykernel_20716\3968032080.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['issue_d'] = pd.to_datetime(data['issue_d'])


In [20]:
data['issue_year'] = data['issue_d'].dt.year
data['issue_month'] = data['issue_d'].dt.month

In [21]:
data['borrower_risk_score'] = (data['debt_to_income'] + data['loan_to_income'] + data['credit_utilization']) / 3

In [22]:
data.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,loan_to_income,interest_rate_category,term_numeric,debt_to_income,payment_to_loan_ratio,credit_utilization,loan_age_months,issue_year,issue_month,borrower_risk_score
392949,39651438,32000.0,32000.0,32000.0,60 months,10.49,687.65,B,B3,Public Service,...,0.266667,Medium,60.0,0.005730,0.021489,686.626298,121.533333,2015,2,228.966232
1273506,16411620,9600.0,9600.0,9600.0,36 months,12.99,323.42,C,C1,Teacher,...,0.438356,Medium,36.0,0.014768,0.033690,115.912596,130.733333,2014,5,38.788574
324024,45122316,4000.0,4000.0,4000.0,36 months,6.68,122.93,A,A3,System Analyst,...,0.048193,Low,36.0,0.001481,0.030733,90.930233,119.566667,2015,4,30.326635
2066630,125356772,6025.0,6025.0,6025.0,36 months,10.91,197.00,B,B4,Admin assistant,...,0.115865,Medium,36.0,0.003788,0.032697,211.406250,87.066667,2017,12,70.508635
477199,128490686,25000.0,25000.0,25000.0,60 months,26.30,752.96,E,E5,Coordinator,...,0.384615,High,60.0,0.011584,0.030118,2002.469636,85.000000,2018,2,667.621945


In [23]:
data = pd.get_dummies(data, columns=['grade', 'sub_grade'], drop_first=True)

In [24]:
data['purpose'] = data['purpose'].fillna('Unknown')

In [25]:
purpose_counts = data['purpose'].value_counts()
data['purpose_grouped'] = data['purpose'].apply(lambda x: x if purpose_counts[x] > 1000 else 'Other')
data = pd.get_dummies(data, columns=['purpose_grouped'], drop_first=True)

In [26]:
data.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_title,emp_length,home_ownership,...,purpose_grouped_credit_card,purpose_grouped_debt_consolidation,purpose_grouped_home_improvement,purpose_grouped_house,purpose_grouped_major_purchase,purpose_grouped_medical,purpose_grouped_moving,purpose_grouped_other,purpose_grouped_small_business,purpose_grouped_vacation
392949,39651438,32000.0,32000.0,32000.0,60 months,10.49,687.65,Public Service,10+ years,MORTGAGE,...,False,True,False,False,False,False,False,False,False,False
1273506,16411620,9600.0,9600.0,9600.0,36 months,12.99,323.42,Teacher,10+ years,RENT,...,False,True,False,False,False,False,False,False,False,False
324024,45122316,4000.0,4000.0,4000.0,36 months,6.68,122.93,System Analyst,4 years,MORTGAGE,...,False,False,False,False,True,False,False,False,False,False
2066630,125356772,6025.0,6025.0,6025.0,36 months,10.91,197.00,Admin assistant,10+ years,RENT,...,False,True,False,False,False,False,False,False,False,False
477199,128490686,25000.0,25000.0,25000.0,60 months,26.30,752.96,Coordinator,10+ years,OWN,...,False,True,False,False,False,False,False,False,False,False


In [27]:
numerical_cols = data.select_dtypes(include=['float64', 'int64']).columns

# Check for infinity
print(data[numerical_cols].isin([float('inf'), float('-inf')]).any())

# Check for NaN values
print(data[numerical_cols].isnull().any())

# Check for very large values
print(data[numerical_cols].describe())


loan_amnt                False
funded_amnt              False
funded_amnt_inv          False
int_rate                 False
installment              False
                         ...  
debt_to_income            True
payment_to_loan_ratio    False
credit_utilization        True
loan_age_months          False
borrower_risk_score       True
Length: 90, dtype: bool
loan_amnt                False
funded_amnt              False
funded_amnt_inv          False
int_rate                 False
installment              False
                         ...  
debt_to_income           False
payment_to_loan_ratio    False
credit_utilization        True
loan_age_months          False
borrower_risk_score       True
Length: 90, dtype: bool
           loan_amnt    funded_amnt  funded_amnt_inv       int_rate  \
count  226070.000000  226070.000000    226070.000000  226070.000000   
mean    15049.306188   15044.313487     15026.783735      13.106018   
std      9179.901864    9178.157889      9181.579904     

In [28]:
data[numerical_cols] = data[numerical_cols].replace([float('inf'), float('-inf')], float('nan'))


In [29]:
# Fill NaN with the median of each column
data[numerical_cols] = data[numerical_cols].fillna(data[numerical_cols].median())


In [30]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
numerical_cols = ['loan_amnt', 'int_rate', 'annual_inc', 'loan_to_income', 'debt_to_income', 'payment_to_loan_ratio']
data[numerical_cols] = scaler.fit_transform(data[numerical_cols])


In [31]:
# Define a function to classify risk levels based on thresholds
def classify_risk(row):
    if row['int_rate'] > 20 or row['loan_to_income'] > 0.5 or row['debt_to_income'] > 0.4:
        return 'High Risk'
    elif 10 < row['int_rate'] <= 20 or 0.3 < row['loan_to_income'] <= 0.5 or 0.2 < row['debt_to_income'] <= 0.4:
        return 'Medium Risk'
    else:
        return 'Low Risk'

# Apply the function to create a new 'risk_level' column
data['risk_level'] = data.apply(classify_risk, axis=1)

# Check the distribution of the risk levels
print(data['risk_level'].value_counts())


risk_level
Low Risk     226066
High Risk         4
Name: count, dtype: int64


C:\Users\91703\AppData\Local\Temp\ipykernel_20716\2673945240.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['risk_level'] = data.apply(classify_risk, axis=1)


In [32]:
data.to_csv('engineered_lending_club_data.csv', index=False)
